# 1. 데이터 불러오기

In [67]:
import os
os.chdir('C:/Users/hu612/Desktop/School/수DA쟁이/Project/개인과제/data')

In [68]:
import pandas as pd
import numpy as np

train=pd.read_csv('train_features.csv')
train_labels=pd.read_csv('train_labels.csv')
test=pd.read_csv('test_features.csv')
submission=pd.read_csv('sample_submission.csv')

In [69]:
train.shape

(1875000, 8)

# 2. 데이터 2차원 변신

In [70]:
import tensorflow as tf
X_train = train[(train['id']>= 0) & (train['id']<=2499)]
X_test = train[train['id'] >= 2500]
X_train = tf.reshape(np.array(X_train.iloc[:,2:]),[-1, 600, 6])
X_test = tf.reshape(np.array(X_test.iloc[:,2:]),[-1, 600, 6])

from tensorflow.keras.utils import to_categorical
y =  train_labels['label']
slice_arr = slice(0,2500,1)
y_train = y[slice_arr]

slice_arr2 = slice(2500,3125,1)
y_test = y[slice_arr2]

y_train_encoding = tf.keras.utils.to_categorical(y_train)
y_test_encoding = tf.keras.utils.to_categorical(y_test)

In [71]:
X_train.shape

TensorShape([2500, 600, 6])

In [72]:
X_train = X_train.numpy()

In [73]:
X_train = X_train.reshape((2500,-1))

In [74]:
X_train.shape

(2500, 3600)

In [75]:
y_train_encoding.shape

(2500, 61)

In [76]:
X_test = X_test.numpy()

In [77]:
X_test.shape

(625, 600, 6)

In [78]:
X_test = X_test.reshape((625,-1))

In [79]:
X_test.shape

(625, 3600)

In [80]:
print(X_train.shape, y_train_encoding.shape)
print(X_test.shape, y_test_encoding.shape)

(2500, 3600) (2500, 61)
(625, 3600) (625, 61)


# 3. RandomForest 해볼까

In [81]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
forest = RandomForestClassifier(n_jobs=-1, random_state=0, min_samples_leaf=30)

In [46]:
forest.fit(X_train, y_train_encoding)

RandomForestClassifier(min_samples_leaf=30, n_jobs=-1, random_state=0)

In [47]:
y_pred = forest.predict(X_test)

In [50]:
y_pred.shape

(625, 61)

In [51]:
print("테스트 세트의 정확도: {:.2f}".format(np.mean(y_pred == y_test_encoding)))

테스트 세트의 정확도: 0.99


In [52]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 6])
test_X.shape

TensorShape([782, 600, 6])

In [53]:
test_X = test_X.numpy()
test_X = test_X.reshape((782,-1))

In [55]:
test_X.shape

(782, 3600)

In [57]:
prediction=forest.predict(test_X)

In [58]:
submission.iloc[:,1:]=prediction

In [59]:
submission.to_csv('C:/Users/hu612/Desktop/School/수DA쟁이/Project/개인과제/data/RANDOMFOREST/RF3.csv', index=False)

# RF3 : 5.312266

In [82]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
forest2 = RandomForestClassifier(random_state=0)

In [83]:
from sklearn.model_selection import GridSearchCV

params ={ 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20] }
grid = GridSearchCV(forest2, param_grid=params, cv=5,n_jobs=-1)
grid.fit(X_train, y_train_encoding)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10, 12],
                         'min_samples_leaf': [8, 12, 18],
                         'min_samples_split': [8, 16, 20],
                         'n_estimators': [10, 100]})

In [85]:
print('최적 하이퍼 파라미터: ', grid.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid.best_score_))

최적 하이퍼 파라미터:  {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도: 0.4444


In [86]:
forest2_1 = RandomForestClassifier(n_estimators = 100, 
                                max_depth = 6,
                                min_samples_leaf = 8,
                                min_samples_split = 8,
                                random_state = 0,
                                n_jobs = -1)
forest2_1.fit(X_train, y_train_encoding)
pred = forest2_1.predict(X_test)
print('예측 정확도: {:.4f}'.format(accuracy_score(y_test_encoding,pred)))

예측 정확도: 0.4592


In [89]:
pred

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [90]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 6])
test_X.shape

TensorShape([782, 600, 6])

In [91]:
test_X = test_X.numpy()
test_X = test_X.reshape((782,-1))

In [93]:
prediction=forest2_1.predict(test_X)

In [94]:
submission.iloc[:,1:]=prediction

In [95]:
submission.to_csv('C:/Users/hu612/Desktop/School/수DA쟁이/Project/개인과제/data/RANDOMFOREST/RF3_1.csv', index=False)

# RF3_1 : 6.4417464